In [1]:
import os
import random
from glob import glob
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torchvision.transforms as T
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

try:
    import faiss
except ImportError:
    faiss = None

SEED = 42
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 100
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

random.seed(SEED)
torch.manual_seed(SEED)

class SimCLRDataset(Dataset):
    def __init__(self, image_paths, transform):
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert('RGB')
        xi = self.transform(img)
        xj = self.transform(img)
        return xi, xj

simclr_transform = T.Compose([
    T.RandomResizedCrop(IMG_SIZE, scale=(0.2, 1.0)),
    T.RandomHorizontalFlip(p=0.5),
    T.ColorJitter(0.8, 0.8, 0.8, 0.2),
    T.RandomGrayscale(p=0.2),
    T.GaussianBlur(kernel_size=9),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

class ProjectionHead(nn.Module):
    def __init__(self, in_dim=2048, hidden_dim=512, out_dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, out_dim)
        )
    def forward(self, x):
        return nn.functional.normalize(self.net(x), dim=1)

class SimCLR(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        self.backbone.fc = nn.Identity()
        # Заморозка первых 4 блоков 
        for param in list(self.backbone.parameters())[:6]:
            param.requires_grad = False
        self.projector = ProjectionHead(2048)
    def forward(self, x):
        h = self.backbone(x)
        z = self.projector(h)
        return z

def nt_xent_loss(z_i, z_j, temperature=0.1):
    z = torch.cat([z_i, z_j], dim=0)
    z = nn.functional.normalize(z, dim=1)
    sim_matrix = torch.matmul(z, z.T) / temperature
    batch_size = z_i.size(0)
    mask = torch.eye(2 * batch_size, dtype=torch.bool).to(DEVICE)
    sim_matrix = sim_matrix.masked_fill(mask, -float('inf'))
    positives = torch.cat([torch.arange(batch_size, 2 * batch_size), torch.arange(0, batch_size)]).to(DEVICE)
    sim_probs = nn.functional.log_softmax(sim_matrix, dim=1)
    loss = -sim_probs[torch.arange(2 * batch_size), positives]
    return loss.mean()

def train_simclr(model, dataloader, optimizer, scheduler=None):
    model.train()
    for epoch in range(EPOCHS):
        total_loss = 0
        for xi, xj in tqdm(dataloader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
            xi, xj = xi.to(DEVICE), xj.to(DEVICE)
            zi, zj = model(xi), model(xj)
            loss = nt_xent_loss(zi, zj)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        if scheduler is not None:
            scheduler.step()
        print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {total_loss/len(dataloader):.4f}")

all_images = glob('/kaggle/input/cow-f-crop/cow_f_crop/*.jpg')
print(f"картинок: {len(all_images)}")

dataset = SimCLRDataset(all_images, simclr_transform)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)

model = SimCLR().to(DEVICE)
LR = 0.3 * BATCH_SIZE / 256
optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

train_simclr(model, dataloader, optimizer, scheduler)

def compute_embeddings(model, paths):
    model.eval()
    embeddings = {}
    transform = T.Compose([
        T.Resize(256),
        T.CenterCrop(IMG_SIZE),
        T.ToTensor(),
        T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    with torch.no_grad():
        for path in tqdm(paths, desc="Embedding"):
            img = Image.open(path).convert('RGB')
            img_tensor = transform(img).unsqueeze(0).to(DEVICE)
            emb = model.backbone(img_tensor).squeeze().cpu().numpy()
            embeddings[os.path.basename(path)] = emb / np.linalg.norm(emb)
    return embeddings

embeddings = compute_embeddings(model, all_images)
np.save("cow_embeddings.npy", embeddings)
print("Эмбеддинги сохранены")

def find_similar(query_img_path, embeddings, top_k=5):
    query_name = os.path.basename(query_img_path)
    query_vector = embeddings[query_name].reshape(1, -1)
    names = list(embeddings.keys())
    all_vectors = np.stack([embeddings[name] for name in names])

    if faiss is not None:
        index = faiss.IndexFlatIP(all_vectors.shape[1])
        index.add(all_vectors.astype('float32'))
        D, I = index.search(query_vector.astype('float32'), top_k+1)
        print(f"\nТоп {top_k} похожих с: {query_name}\n")
        for i, idx in enumerate(I[0][1:top_k+1]):
            print(f"{names[idx]} (sim: {D[0][i+1]:.4f})")
    else:
        sims = cosine_similarity(query_vector, all_vectors)[0]
        top_idxs = sims.argsort()[::-1][1:top_k+1]
        print(f"\nТоп {top_k} похожих с: {query_name}\n")
        for idx in top_idxs:
            print(f"{names[idx]} (sim: {sims[idx]:.4f})")

картинок: 315


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 201MB/s]
Epoch 1/100: 100%|██████████| 4/4 [00:07<00:00,  1.89s/it]


Epoch 1/100, Loss: 4.8945


Epoch 2/100: 100%|██████████| 4/4 [00:06<00:00,  1.55s/it]


Epoch 2/100, Loss: 4.6826


...

Epoch 99/100, Loss: 0.8889


Epoch 100/100: 100%|██████████| 4/4 [00:06<00:00,  1.57s/it]


Epoch 100/100, Loss: 0.9276


Embedding: 100%|██████████| 315/315 [00:05<00:00, 62.35it/s]

Эмбеддинги сохранены

Топ 5 похожих с: processed_1_79d4e30a-IMG_20221011_103356.jpg

processed_1_a8cbf437-IMG_20221011_103352.jpg (sim: 0.9157)
processed_51_9d315f77-IMG_20221011_123545.jpg (sim: 0.8278)
processed_57_83ee8236-IMG_20221011_125037.jpg (sim: 0.8176)
processed_36_d557aca7-IMG_20221011_120343.jpg (sim: 0.7204)
processed_30_d9137b1a-IMG_20221011_115024.jpg (sim: 0.6897)


In [2]:
find_similar('/kaggle/input/cow-f-crop/cow_f_crop/processed_1_79d4e30a-IMG_20221011_103356.jpg', embeddings, top_k=1)


Топ 1 похожих с: processed_1_79d4e30a-IMG_20221011_103356.jpg

processed_1_a8cbf437-IMG_20221011_103352.jpg (sim: 0.9157)


In [5]:
find_similar('/kaggle/input/cow-f-crop/cow_f_crop/processed_23_69024848-IMG_20221011_112623.jpg', embeddings, top_k=1)


Топ 1 похожих с: processed_23_69024848-IMG_20221011_112623.jpg

processed_23_8dad22cb-IMG_20221011_112624.jpg (sim: 0.9042)


In [8]:
find_similar('/kaggle/input/cow-f-crop/cow_f_crop/processed_27_a94a8a5e-IMG_20221011_113859.jpg', embeddings, top_k=1)


Топ 1 похожих с: processed_27_a94a8a5e-IMG_20221011_113859.jpg

processed_27_e0939cc5-IMG_20221011_113858.jpg (sim: 0.9660)


In [9]:
torch.save({
    'model_state_dict': model.state_dict(),
    'model_architecture': 'SimCLR',
    'img_size': IMG_SIZE,
    'backbone': 'resnet50'
}, 'simclr_cow_model.pth')
print("Модель сохранена: simclr_cow_model.pth")

Модель сохранена: simclr_cow_model.pth
